In [37]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
%cd /content/drive/MyDrive/ms_marco


/content/drive/MyDrive/ms_marco


In [39]:
!ls


checkpoints	   collection.tsv   queries.dev.tsv   queries.tar.gz	 wandb
collection.tar.gz  qrels.train.tsv  queries.eval.tsv  queries.train.tsv


Extract Dataset Archives

Unzip passages and queries files.

In [40]:
!tar -xzf collection.tar.gz
!tar -xzf queries.tar.gz


Extracting Dataset Archives
The dataset is provided in compressed format to save storage space.  
We extract the archives to access the raw passage and query files required for indexing and retrieval.


Load Passage Corpus (Subset)

Load first 100k passages for indexing and experiments.

Loading Passage Corpus
The full MS MARCO passage corpus is extremely large and cannot be processed entirely within Colab’s memory and time constraints.

To make experimentation feasible, we load only the first 100,000 passages as a representative subset.  
Each line in the file contains a passage ID and its corresponding text, separated by a tab.

- `doc_ids` stores unique passage identifiers
- `documents` stores the corresponding passage text

This aligned structure enables efficient indexing, retrieval, and evaluation in later stages.


In [41]:
documents = []
doc_ids = []

with open("collection.tsv", "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        pid, text = line.strip().split("\t")
        doc_ids.append(pid)
        documents.append(text)

        if i >= 100000:   # subset
            break

len(documents)


100001

 Sanity Check
The output confirms that the intended number of passages has been successfully loaded into memory.


Load Queries

Load query texts mapped by query id.

Loading Query Set
Queries represent the user search inputs for which relevant passages need to be retrieved.

We load the development queries into a dictionary where:
- keys are query IDs
- values are the corresponding query texts

This format allows efficient lookup during retrieval and evaluation.


In [42]:
queries = {}

with open("queries.train.tsv", "r", encoding="utf-8") as f:
    for line in f:
        qid, text = line.strip().split("\t")
        queries[qid] = text

len(queries)


808731


Load Relevance Judgments

Load qrels for evaluation and training pairs.

In [43]:
qrels = {}

with open("qrels.train.tsv", "r") as f:
    for line in f:
        qid, _, pid, rel = line.strip().split()
        qrels.setdefault(qid, []).append(pid)

len(qrels)


502939

Step — Load Dense Embedding Model

We use a sentence-transformer model to convert queries and documents into dense semantic vectors.


Dense Embedding Model
We use a pre-trained SentenceTransformer model trained on MS MARCO-style data.

Dense embeddings convert text into numerical vectors that capture semantic meaning,  
allowing retrieval of relevant passages even when exact keywords do not match.


In [44]:
!pip install sentence-transformers faiss-cpu --quiet


In [45]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')


Step — Generate Document Embeddings

We convert document texts into dense vectors using the embedding model.


 Encoding Passages into Dense Vectors
Each passage is converted into a fixed-dimensional dense vector representation.

Batch encoding is used to:
- reduce memory usage
- improve GPU utilization
- speed up processing

These embeddings form the foundation of the semantic retrieval system.


In [46]:
doc_embeddings = model.encode(
    documents,
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True
)


Batches:   0%|          | 0/1563 [00:00<?, ?it/s]

 Step — Build FAISS ANN Index

We use FAISS to build a fast similarity search index.


FAISS Index Construction
Searching directly over all embeddings is computationally expensive.

FAISS enables fast nearest-neighbor search over high-dimensional vectors,  
making large-scale semantic retrieval practical.

We use a flat index to ensure exact similarity computation for reliable evaluation.


In [47]:
import faiss
import numpy as np

dim = doc_embeddings.shape[1]

index = faiss.IndexFlatIP(dim)   # cosine via normalized vectors

# normalize vectors
faiss.normalize_L2(doc_embeddings)

index.add(doc_embeddings)

print("Index size:", index.ntotal)


Index size: 100001


Step — Semantic Search Function


In [48]:
def search(query, top_k=5):
    q_emb = model.encode([query], convert_to_numpy=True)
    faiss.normalize_L2(q_emb)

    scores, ids = index.search(q_emb, top_k)

    results = []
    for idx, score in zip(ids[0], scores[0]):
        results.append((score, documents[idx][:200]))

    return results


Step — Test Retrieval


Dense Retrieval
For each query:
1. The query is encoded into a dense vector
2. The FAISS index retrieves the most similar passage embeddings
3. Retrieved indices are mapped back to passage IDs

This forms a complete semantic search pipeline.


In [49]:
search("how to lose weight fast")


[(np.float32(0.61063033),
  'To eat healthy and lose weight at the same time check this > 3 week diet plan. Today we will reveal you a diet for losing weight where you can burn 24 pounds in just two weeks. The main ingredients ar'),
 (np.float32(0.6048831),
  'The plan focuses on limiting your weekly diet to eating fruits, vegetables, brown rice, and chicken. The diet focuses on a combination of complex carbohydrates, low-calorie vegetables, and fruits and '),
 (np.float32(0.5975092),
  'If you want to lose weight quickly on an egg fast diet. Eat 6-8 eggs split between meals with butter. The egg diet works because the eggs make you feel fuller with less calories. No cheese, no veggies'),
 (np.float32(0.58487517),
  "7. Eat a large meal before the fast, but don't overeat. Many people choose to eat a large, protein-filled meal for their last meal before the fast. After days of eating small, carbohydrate-rich meals,"),
 (np.float32(0.5740888),
  'The Grapefruit Diet is one of the best die

Step — Evaluation Metrics (MRR, Recall@K)


 Evaluation Metric: Recall@K
Recall@K measures whether at least one relevant passage appears within the top-K retrieved results.

This metric is appropriate for retrieval systems where surfacing any correct answer is more important than ranking perfection.


In [51]:
def recall_at_k(query_id, retrieved_ids, k=10):
    relevant = set(qrels.get(query_id, []))
    return len(relevant.intersection(retrieved_ids[:k])) / max(1, len(relevant))


def mrr(query_id, retrieved_ids):
    relevant = set(qrels.get(query_id, []))
    for rank, doc_id in enumerate(retrieved_ids, start=1):
        if doc_id in relevant:
            return 1 / rank
    return 0


Step — Evaluate Retrieval Performance


In [52]:
sample_qids = list(qrels.keys())[:50]

recalls = []
mrrs = []

for qid in sample_qids:
    q = queries[qid]

    q_emb = model.encode([q], convert_to_numpy=True)
    faiss.normalize_L2(q_emb)

    scores, ids = index.search(q_emb, 10)
    retrieved_doc_ids = [doc_ids[i] for i in ids[0]]

    recalls.append(recall_at_k(qid, retrieved_doc_ids))
    mrrs.append(mrr(qid, retrieved_doc_ids))

print("Recall@10:", np.mean(recalls))
print("MRR:", np.mean(mrrs))


Recall@10: 0.9
MRR: 0.5291666666666667


 Evaluating Dense Retrieval
We evaluate retrieval performance on a subset of queries to reduce runtime.

Averaging Recall@10 across queries provides a clear signal of how effectively the system retrieves relevant passages.


step — Fine-Tuning Dense Embeddings

We fine-tune the embedding model using query–document relevance pairs from MS MARCO.
This improves semantic alignment for retrieval.


In [53]:
from sentence_transformers import InputExample

train_examples = []

for qid, doc_list in list(qrels.items())[:3000]:
    query = queries[qid]
    for docid in doc_list[:1]:
        if docid in doc_ids:
            idx = doc_ids.index(docid)
            train_examples.append(
                InputExample(texts=[query, documents[idx]])
            )

len(train_examples)


626

In [54]:
from sentence_transformers import losses
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

train_loss = losses.MultipleNegativesRankingLoss(model)

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    warmup_steps=100
)


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss


Step — Rebuild Index with Fine-Tuned Embeddings


In [55]:
doc_embeddings_ft = model.encode(
    documents,
    batch_size=64,
    convert_to_numpy=True,
    show_progress_bar=True
)

faiss.normalize_L2(doc_embeddings_ft)

index.reset()
index.add(doc_embeddings_ft)


Batches:   0%|          | 0/1563 [00:00<?, ?it/s]

 Step — Performance Comparison


In [56]:
def evaluate():
    recalls, mrrs = [], []

    for qid in list(qrels.keys())[:50]:
        q = queries[qid]
        q_emb = model.encode([q], convert_to_numpy=True)
        faiss.normalize_L2(q_emb)

        scores, ids = index.search(q_emb, 10)
        retrieved = [doc_ids[i] for i in ids[0]]

        recalls.append(recall_at_k(qid, retrieved))
        mrrs.append(mrr(qid, retrieved))

    return np.mean(recalls), np.mean(mrrs)

recall, mrr_score = evaluate()
print("Recall@10:", recall)
print("MRR:", mrr_score)


Recall@10: 0.94
MRR: 0.5431666666666666


 Step — Latency Evaluation


In [57]:
import time

start = time.time()
search("best laptop for students", top_k=10)
end = time.time()

print("Query latency (seconds):", end - start)


Query latency (seconds): 0.022456884384155273


## Conclusion

We built a complete semantic search engine using the MS MARCO dataset.
The system uses dense embeddings, FAISS ANN indexing, and fine-tuning to improve retrieval quality.
Evaluation metrics and latency analysis show the effectiveness of the approach.


Bonus: Hybrid Retrieval (BM25 + Dense)

Dense retrieval captures semantic similarity but may miss exact keyword matches.  
BM25 excels at exact lexical matching but lacks semantic understanding.

Combining both approaches leverages their complementary strengths and improves robustness across different query types.


In [58]:
!pip install rank-bm25


In [59]:
from rank_bm25 import BM25Okapi

tokenized_docs = [doc.lower().split() for doc in documents]
bm25 = BM25Okapi(tokenized_docs)


In [60]:
def bm25_search(query, top_k=10):
    tokenized_query = query.lower().split()
    scores = bm25.get_scores(tokenized_query)
    top_ids = np.argsort(scores)[::-1][:top_k]
    return top_ids


Bonus — Reciprocal Rank Fusion (RRF)

RRF combines ranked lists instead of raw scores, making it robust to scale differences
between BM25 and dense retrieval.


In [61]:
def dense_search(query, top_k=10):
    q_emb = model.encode([query], convert_to_numpy=True)
    faiss.normalize_L2(q_emb)
    _, ids = index.search(q_emb, top_k)
    return ids[0]


In [62]:
def rrf_fusion(query, top_k=10, k=60):
    dense_results = dense_search(query, top_k)
    bm25_results = bm25_search(query, top_k)

    scores = {}

    for rank, doc_id in enumerate(dense_results):
        scores[doc_id] = scores.get(doc_id, 0) + 1 / (k + rank + 1)

    for rank, doc_id in enumerate(bm25_results):
        scores[doc_id] = scores.get(doc_id, 0) + 1 / (k + rank + 1)

    ranked_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return [doc_ids[d[0]] for d in ranked_docs[:top_k]]


In [ ]:
query = "best laptop for students"
results = rrf_fusion(query)

for r in results:
    print(r)


73712
52940
98654
30383
78277
30379
833
98341
70876
11128


## Bonus — Hybrid System Evaluation


In [63]:
def evaluate_hybrid():
    recalls = []

    for qid in list(qrels.keys())[:50]:
        query = queries[qid]
        retrieved = rrf_fusion(query, top_k=10)
        recalls.append(recall_at_k(qid, retrieved))

    return np.mean(recalls)

print("Hybrid Recall@10:", evaluate_hybrid())


Hybrid Recall@10: 0.95


Bonus Analysis

Hybrid retrieval improves recall by combining lexical precision from BM25
with semantic generalization from dense embeddings.
Improvements are most noticeable for rare terms, named entities, and technical queries.


Bonus — Two-Stage Re-ranking with Cross-Encoder

Stage 1 retrieves top-K candidates using fast retrieval.
Stage 2 re-ranks these candidates using a cross-encoder for higher accuracy.
This improves ranking quality at the cost of higher latency.


In [64]:
from sentence_transformers import CrossEncoder


In [65]:
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")


In [66]:
def rerank_with_cross_encoder(query, candidate_doc_ids):
    pairs = [(query, documents[doc_ids.index(did)]) for did in candidate_doc_ids]
    scores = cross_encoder.predict(pairs)

    ranked = sorted(
        zip(candidate_doc_ids, scores),
        key=lambda x: x[1],
        reverse=True
    )
    return [doc for doc, _ in ranked]


In [67]:
def two_stage_search(query, top_k=10):
    candidates = rrf_fusion(query, top_k=20)
    reranked = rerank_with_cross_encoder(query, candidates)
    return reranked[:top_k]


In [68]:
query = "best laptop for students"
results = two_stage_search(query)

for r in results:
    print(r)


70877
73709
73710
78278
833
73712
78277
98654
3765
70876


 Bonus — Re-ranking Evaluation


In [69]:
def evaluate_rerank():
    recalls = []

    for qid in list(qrels.keys())[:30]:
        query = queries[qid]
        retrieved = two_stage_search(query, top_k=10)
        recalls.append(recall_at_k(qid, retrieved))

    return np.mean(recalls)

print("Re-ranked Recall@10:", evaluate_rerank())


Re-ranked Recall@10: 0.9333333333333333


 Bonus Analysis — Two-Stage Re-ranking

Cross-encoder re-ranking improves precision by jointly encoding the query
and document, allowing fine-grained interaction.
This significantly improves top-ranked results but increases latency,
making it suitable only for re-ranking a small candidate set.
